In [21]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from sklearn import *
import random
import pickle
import math

In [9]:
class Preprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter
	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [10]:
class SimplePreprocessorGray:

	def preprocess(self, image):
		return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )

In [11]:
# Classe para carregar a base de imagens
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None):
    # store the image preprocessor
    self.preprocessors = preprocessors
		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []
		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      			# check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)
			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)

      # show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        print("[INFO] processed {}/{}".format(i + 1,
					len(imagePaths)))
		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

In [12]:
# Definindo os parametros
class Parametros:
  dataset = "./Imagens"
  neighbors = 1
  jobs = -1

args = vars(Parametros)

In [13]:
# grab the list of images that we'll be describing
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = Preprocessor(32, 32)
spg = SimplePreprocessorGray()
sdl = SimpleDatasetLoader(preprocessors=[spg])
(data, labels) = sdl.load(imagePaths, verbose=500)
#data = data.reshape((data.shape[0], 60200))

# show some information on memory consumption of the images
print("[INFO] features matrix: {:.1f}MB".format(
	data.nbytes / (1024 * 1024.0)))

[INFO] loading images...
[INFO] features matrix: 2.5MB


In [28]:
######################### exploração para encontrar parametros mais performàticos para modelo #######################
accuracy_list = [0]
vocab_size_percentage = 10
test_size = 0.25
n_neighbors = 3
try_numbers = 100

for i in range(try_numbers):
  if (i % 5 == 0):
    vocab_size_percentage = random.randint(10, 25)
    test_size = random.uniform(0.15, 0.40)
    random_n_neighbors = random.randint(1, 99)

    if random_n_neighbors > 66:
      n_neighbors = 3
    elif random_n_neighbors > 33:
      n_neighbors = 5
    else:
      n_neighbors = 7

     
  # Obtendo keypoints das imagens
  sift = cv2.SIFT_create()
  principais_caracteristicas = []
  principais_labels = []
  n_sample = 50 # 50 keypoints de cada img pra tbm nao ficar pesado

  for idx, img in enumerate(data):
    # pega o sift de cada img
    keypoints, descriptors = sift.detectAndCompute(data[idx], None)

    principais = descriptors[np.random.randint(descriptors.shape[0], size=n_sample)]

    principais_caracteristicas[(idx*n_sample):(idx*n_sample)+n_sample] = principais[:]

  vocab_size = len(principais_caracteristicas) // vocab_size_percentage # Vocabulario vai ter 10% das keypoints
  vocab = KMeans(n_clusters=2).fit(principais_caracteristicas, vocab_size)

  # Obtendo histogramas da base de imgs X vocabulario das principais keypoints

  histo_list = []

  # For cada imagem
  for idx, img in enumerate(data):
    # Obtenho novamente os keypoints da img
    keypoints, descriptors = sift.detectAndCompute(data[idx], None)

    # Verifico como o vocabulario enquadra eles, montando assim o histograma
    histo = np.zeros(n_sample)
    nkp = np.size(keypoints)

    for d in descriptors:
        idx = vocab.predict([d])
        histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

    histo_list.append(histo)

  # transforma as categorias em inteiros
  le = LabelEncoder()
  labels_number = le.fit_transform(labels)

  trainX, testX, trainY, testY = train_test_split(histo_list, labels_number, test_size=test_size, random_state=42)

  # Com esses histogramas, eu treino o meu modelo

  knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
  knn_model.fit(trainX, trainY)

  knn_model_accuracy = float(classification_report(testY, knn_model.predict(testX))[204:207])

  loading_filled = ['=' for _ in range(i)]
  loading_blank = [' ' for _ in range(try_numbers-i)]
  loading = loading_filled + loading_blank
  loading[math.floor(try_numbers/2)] = str(i)
  loading.insert(0, '[')
  loading.append(']')
  print(''.join(loading))

  if accuracy_list[i] < knn_model_accuracy:
    accuracy_list.append(knn_model_accuracy)
    print(f'model accuracy - {knn_model_accuracy}')
    print(f'vocab_size_percentage={vocab_size_percentage}')
    print(f'test_size={test_size}')
    print(f'n_neighbors={n_neighbors}')
    
    pickle.dump(vocab, open("MontadorHistogramaTESTE.pkl", "wb"))
    pickle.dump(knn_model, open("LeitorHistogramaTESTE.pkl", "wb"))
  else:
    accuracy_list.insert(0, knn_model_accuracy)
  

C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[                                                  0                                                 ]
model accuracy - 0.67
vocab_size_percentage=14
test_size=0.20175219872759673
n_neighbors=5


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[=                                                 1                                                 ]


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

[==================================================97==============================================   ]


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[==================================================98===============================================  ]


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[==================================================99================================================ ]
